**EJERCICIO PRÁCTICO - ANALÍTICA EN SALUD: CANTIDAD DE CIRUGÍAS**

In [ ]:
# Bases de datos a usar:
# 1. RETO_df_egresos -> Base de egresos hospitalarios (no aplica en nuestro caso).
# 2. RETO_df_cx -> Base de cirugías.
# 3. RETO_df_cronicos -> Base de datos crónico
# 4. RETO_df_usuarios -> Base de datos con la población mensualmente.

Importar Bases de datos

In [ ]:
#Conexión con colab
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from unicodedata import normalize

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Leer cada una de las bases de datos.
cx = pd.read_csv('/content/drive/MyDrive/CursoAnalitica3/AplicacionesEnSalud/RETO_df_cx.csv')
cronicos = pd.read_csv('/content/drive/MyDrive/CursoAnalitica3/AplicacionesEnSalud/RETO_df_cronicos.csv')
usuarios = pd.read_csv('/content/drive/MyDrive/CursoAnalitica3/AplicacionesEnSalud/RETO_df_usuarios.csv')
egresos = pd.read_csv('/content/drive/MyDrive/CursoAnalitica3/AplicacionesEnSalud/RETO_df_egresos.csv')

In [ ]:
cx.shape

(2370, 77)

In [ ]:
cx.head(5)

,YEAR,MES,CODIGO SERVICIO HABILITACION,SERVICIO HABILITACION,TIPO IDENTIFICACION,NRODOC,FECHA NACIMIENTO,REGIMEN,CODIGO EPS,DESCRIPCION EPS VALIDADA,...,COD DIAGNOSTICO POSTQUIRURGICO,DIAGNOSTICO POSTQUIRURGICO,TIPO DE HERIDA,TIPO AISLAMIENTO,COMPLICACIONES,CATEGORIA COMPLICACION,ESPECIALIDAD,SEDE,IDCX,ADMISION
0,2017,ABRIL,203.0,Cirugía General,CC,306809,NaN,NaN,NaN,NaN,...,NaN,NaN,Sucia,NaN,NaN,NaN,Cirugia General,LEON XIII,NaN,3395320
1,2017,ABRIL,203.0,Cirugía General,CC,306809,NaN,NaN,NaN,NaN,...,NaN,NaN,Sucia,NaN,NaN,NaN,Cirugia General,LEON XIII,NaN,3395320
2,2017,ABRIL,203.0,Cirugía General,CC,306809,NaN,NaN,NaN,NaN,...,NaN,NaN,Sucia,NaN,NaN,NaN,Cirugia General,LEON XIII,NaN,3395320
3,2017,ABRIL,203.0,Cirugía General,CC,306809,NaN,NaN,NaN,NaN,...,NaN,NaN,Sucia,NaN,NaN,NaN,Cirugia General,LEON XIII,NaN,3395320
4,2017,ABRIL,203.0,Cirugía General,CC,306809,NaN,NaN,NaN,NaN,...,NaN,NaN,Sucia,NaN,NaN,NaN,Cirugia General,LEON XIII,NaN,3395320


In [ ]:
cx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2370 entries, 0 to 2369
Data columns (total 77 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   YEAR                            2370 non-null   int64  
 1   MES                             2370 non-null   object 
 2   CODIGO SERVICIO HABILITACION    1988 non-null   float64
 3   SERVICIO HABILITACION           1988 non-null   object 
 4   TIPO IDENTIFICACION             2370 non-null   object 
 5   NRODOC                          2370 non-null   int64  
 6   FECHA NACIMIENTO                2282 non-null   object 
 7   REGIMEN                         1851 non-null   object 
 8   CODIGO EPS                      1580 non-null   float64
 9   DESCRIPCION EPS VALIDADA        1087 non-null   object 
 10  FECHA ORDEN                     2364 non-null   object 
 11  HORA ORDEN                      1087 non-null   object 
 12  ORIGEN ORDEN                    15

1. Depuración BD Cirugías (cx)

In [ ]:
# Eliminamos las variables en donde más del 63% de sus datos son nulos, tales como:
cx = cx.drop(['CODCUPS1', 'PROCEDIMIENTOCIRUGIA1', 'CODCUPS2', 'PROCEDIMIENTOCIRUGIA2', 'CODCUPS3', 'PROCEDIMIENTOCIRUGIA3',
              'CODCUPS4', 'PROCEDIMIENTOCIRUGIA4', 'CODCUPS5', 'PROCEDIMIENTOCIRUGIA5', 'CODCUPS6', 'PROCEDIMIENTOCIRUGIA6',
              'CODCUPS7', 'PROCEDIMIENTOCIRUGIA7', 'CODCUPS8', 'PROCEDIMIENTOCIRUGIA8', 'CODCUPS9', 'PROCEDIMIENTOCIRUGIA9',
              'CODCUPS10', 'PROCEDIMIENTOCIRUGIA10', 'BLOQUE', 'PISO', 'SERVICIO', 'BLOQUE DESTINO', 'CAMA DESTINO', 'PISO DESTINO',
              'SERVICIO DESTINO'], axis = 1)

In [ ]:
cx.shape

(2370, 50)

In [ ]:
print(cx[['HORA ORDEN']].isnull().sum())
print(cx[['HORA AUDITA ORDEN']].isnull().sum())

HORA ORDEN    1283
dtype: int64
HORA AUDITA ORDEN    1419
dtype: int64


In [ ]:
print(cx[['FECHA ORDEN']].isnull().sum())
print(cx[['FECHA AUDITA ORDEN']].isnull().sum())

FECHA ORDEN    6
dtype: int64
FECHA AUDITA ORDEN    814
dtype: int64


In [ ]:
# Cuando se hace la comparación de fecha orden vs fecha audita orden se observa que la última tiene gran cantidad de nulos.
# Además, los registros que tienen en común son la misma fecha, por ende, se decide eliminar FECHA AUDITA ORDEN.
# Lo mismo pasa con la hora orden vs hora audita orden, las horas son semejantes y por un inicio no creemos necesaria
# dicha variable, por ende se decide eliminar ambas.
# Otras variables a eliminar: ASA porque no tiene congruencia con la clasificación del diccionario, HORA CIRUGIA - HORA ENTRADA QX,
# HORA INICIO ANESTESIA, HORA INICIO CX, HORA FIN CX, HORA FIN ANESTECIA, HORA SALIDA CX porque nuestro tema a tratar es cantidad 
# de cirugías, mas no tiempo quirúrgico.
cx = cx.drop(['FECHA AUDITA ORDEN', 'HORA AUDITA ORDEN', 'HORA ORDEN', 'HORA CIRUGIA', 'HORA ENTRADA QX', 'HORA INICIO ANESTESIA',
              'HORA INICIO CX', 'HORA FIN CX', 'HORA FIN CX', 'HORA FIN ANESTESIA', 'HORA SALIDA QX'], axis =1)

In [ ]:
cx.shape

(2370, 40)

In [ ]:
# Convertimos las fechas en formato datetime
cx['FECHA NACIMIENTO'] = pd.to_datetime(cx['FECHA NACIMIENTO'])
cx['FECHA ORDEN'] = pd.to_datetime(cx['FECHA ORDEN'])
cx['FECHA CIRUGIA'] = pd.to_datetime(cx['FECHA CIRUGIA'])
cx['FECHA RECUPERACION CX'] = pd.to_datetime(cx['FECHA RECUPERACION CX'])

In [ ]:
# Aplicar funciones para homologar a nivel general.
lista = ['SERVICIO HABILITACION', 'REGIMEN', 'DESCRIPCION EPS VALIDADA',
       'ORIGEN ORDEN', 'ORIGEN ADMSION', 'QUIROFANO', 'TIPO PROCEDIMIENTO',
       'TIPO DE ANESTESIA', 'VIA',
       'TIPO DE HERIDA', 'ESPECIALIDAD']
for i in lista:
    cx[i] = cx[i].str.lower() #minuscula
    cx[i] = cx[i].str.strip() #espacios
    cx[i] = cx[i].apply(lambda x: normalize("NFKD", x).encode("ascii", "ignore").decode("ascii") if pd.notnull(x) else x)

In [ ]:
# Imprimimos las categorías de las variables
lista = ['SERVICIO HABILITACION', 'REGIMEN', 'DESCRIPCION EPS VALIDADA',
       'ORIGEN ORDEN', 'ORIGEN ADMSION', 'QUIROFANO', 'TIPO PROCEDIMIENTO',
       'TIPO DE ANESTESIA', 'VIA',
       'TIPO DE HERIDA', 'ESPECIALIDAD']
for i in lista:
    print(cx[i].value_counts())
    print('------------------------------------')

cirugia ortopedica                           620
cirugia general                              401
otras cirugias                               284
cirugia urologica                            153
cirugia vascular y angiologica               150
cirugia plastica y estetica                  126
cirugia ginecologica                          48
cirugia otorrinolaringologia                  39
cirugia de torax                              37
cirugia neurologica                           35
neurocirugia                                  19
cirugia gastrointestinal                      18
cirugia cardiovascular                        12
cirugia oncologica                            11
cirugia endovascular neurologica              10
cirugia general (cirugia cabeza y cuello)      8
cirugia maxilofacial                           8
trasplante de tejido osteomuscular             2
cirugia oftalmologica                          2
trasplante osteomuscular                       2
cirugia de cabeza y 

In [ ]:
# Homologar los valores de las anteriores variables
cx['SERVICIO HABILITACION'] = cx['SERVICIO HABILITACION'].replace(['cirugia general (cirugia cabeza y cuello)', 'cirugia de cabeza y cuello'], 'cirugia general')
cx['SERVICIO HABILITACION'] = cx['SERVICIO HABILITACION'].replace(['cirugia maxilofacial', 'trasplante de tejido osteomuscular', 'cirugia oftalmologica', 'rasplante osteomuscular', 'trasplante tejidos cardiovasculares'], 'otras cirugias')
cx['REGIMEN'] = cx['REGIMEN'].replace({'no aplica': 'contributivo'})
cx['DESCRIPCION EPS VALIDADA'] = cx['DESCRIPCION EPS VALIDADA'].replace(['nueva eps s.a. pgp', 'no afiliado particular', 'ensayo clinico', 'axa seguros colpatria s.a'], 'nueva eps s.a.')
cx['QUIROFANO'] = cx['QUIROFANO'].replace(['sala 1', 'quirofano 1'], '1')
cx['QUIROFANO'] = cx['QUIROFANO'].replace(['sala 2', 'quirofano 2'], '2')
cx['QUIROFANO'] = cx['QUIROFANO'].replace(['sala 3', 'quirofano 3'], '3')
cx['QUIROFANO'] = cx['QUIROFANO'].replace(['sala 4', 'quirofano 4'], '4')
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 5': '5'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 6': '6'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 7': '7'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 8': '8'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 9': '9'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 10': '10'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 11': '11'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 12': '12'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 13': '13'})
cx['QUIROFANO'] = cx['QUIROFANO'].replace({'sala 14': '14'})
cx['TIPO PROCEDIMIENTO'] = cx['TIPO PROCEDIMIENTO'].replace({'quirurgicas': 'quirurgico'})
cx['TIPO PROCEDIMIENTO'] = cx['TIPO PROCEDIMIENTO'].replace({'quirurgicos': 'quirurgico'})
cx['TIPO PROCEDIMIENTO'] = cx['TIPO PROCEDIMIENTO'].replace({'cirugias': 'quirurgico'})
cx['TIPO DE HERIDA'] = cx['TIPO DE HERIDA'].replace(['no registra', 'no aplica'], 'sin informe quirurgico')
cx['ESPECIALIDAD'] = cx['ESPECIALIDAD'].replace(['cirugia gastroenterologia - cpre', 'gineco oncologia',
       'cirugia cardiovascular', 'neumologia', 'medicina general', 'hepatobiliopancreatica', 'fisiatria',
       'cirugia de cabeza y cuello', 'cirugia oncologica', 'cirugia maxilofacial', 'neurologia',
       'cirugia hepatobiliopancreatica', 'odontologia', 'gine oncologia', 'oftalmologia', 'cirugia de trasplante',
       'ortopedia oncologica'], 'otras cirugias')

In [ ]:
# Imprimimos las categorías de las variables de nuevo para corrobar que se hayan homologado
lista = ['SERVICIO HABILITACION', 'REGIMEN', 'DESCRIPCION EPS VALIDADA',
       'ORIGEN ORDEN', 'ORIGEN ADMSION', 'QUIROFANO', 'TIPO PROCEDIMIENTO',
       'TIPO DE ANESTESIA', 'VIA',
       'TIPO DE HERIDA', 'ESPECIALIDAD']
for i in lista:
    print(cx[i].value_counts())
    print('------------------------------------')

cirugia ortopedica                  620
cirugia general                     411
otras cirugias                      297
cirugia urologica                   153
cirugia vascular y angiologica      150
cirugia plastica y estetica         126
cirugia ginecologica                 48
cirugia otorrinolaringologia         39
cirugia de torax                     37
cirugia neurologica                  35
neurocirugia                         19
cirugia gastrointestinal             18
cirugia cardiovascular               12
cirugia oncologica                   11
cirugia endovascular neurologica     10
trasplante osteomuscular              2
Name: SERVICIO HABILITACION, dtype: int64
------------------------------------
subsidiado      1438
contributivo     413
Name: REGIMEN, dtype: int64
------------------------------------
nueva eps s.a.    1087
Name: DESCRIPCION EPS VALIDADA, dtype: int64
------------------------------------
ambulatorio          1038
hospitalizacion       380
urgencias adultos

In [ ]:
# Dividimos la BD teniendo en cuenta post-pandemia (2021 y 2022)
cx.head(2)

,YEAR,MES,CODIGO SERVICIO HABILITACION,SERVICIO HABILITACION,TIPO IDENTIFICACION,NRODOC,FECHA NACIMIENTO,REGIMEN,CODIGO EPS,DESCRIPCION EPS VALIDADA,...,COD DIAGNOSTICO POSTQUIRURGICO,DIAGNOSTICO POSTQUIRURGICO,TIPO DE HERIDA,TIPO AISLAMIENTO,COMPLICACIONES,CATEGORIA COMPLICACION,ESPECIALIDAD,SEDE,IDCX,ADMISION
0,2017,ABRIL,203.0,cirugia general,CC,306809,NaT,NaN,NaN,NaN,...,NaN,NaN,sucia,NaN,NaN,NaN,cirugia general,LEON XIII,NaN,3395320
1,2017,ABRIL,203.0,cirugia general,CC,306809,NaT,NaN,NaN,NaN,...,NaN,NaN,sucia,NaN,NaN,NaN,cirugia general,LEON XIII,NaN,3395320


In [ ]:
# Base de datos Cirugías Post Pandemia
cxpost = cx.drop(cx[cx['YEAR']<2020].index)
cxpost.tail(3)

,YEAR,MES,CODIGO SERVICIO HABILITACION,SERVICIO HABILITACION,TIPO IDENTIFICACION,NRODOC,FECHA NACIMIENTO,REGIMEN,CODIGO EPS,DESCRIPCION EPS VALIDADA,...,COD DIAGNOSTICO POSTQUIRURGICO,DIAGNOSTICO POSTQUIRURGICO,TIPO DE HERIDA,TIPO AISLAMIENTO,COMPLICACIONES,CATEGORIA COMPLICACION,ESPECIALIDAD,SEDE,IDCX,ADMISION
2367,2022,JUNIO,213.0,cirugia plastica y estetica,CC,307306,1936-10-15,contributivo,3289.0,nueva eps s.a.,...,C443,TUMOR MALIGNO DE LA PIEL DE OTRAS PARTES Y DE ...,limpia,NaN,False,2.0,cirugia plastica,PRADO,132700.0,6129844
2368,2022,JUNIO,203.0,cirugia general,CC,307270,1956-12-22,contributivo,1791.0,nueva eps s.a.,...,K439,OTRAS HERNIAS VENTRALES Y LAS NO ESPECIFICADAS...,limpia,NaN,False,2.0,cirugia general,PRADO,132904.0,6136389
2369,2022,JUNIO,NaN,NaN,CC,313523,1979-05-11,contributivo,1791.0,nueva eps s.a.,...,L600,UNA ENCARNADA,limpia contaminada,NaN,False,1.0,dermatologia,PRADO,133370.0,6150985


In [ ]:
# Dimensión CX Post Pandemia
cxpost.shape

(1087, 40)

In [ ]:
# Descargar la BD depurada
cxpost.to_csv('cxpost.csv')